# Violations of ETH

$
\require{physics}
\def\bm{\boldsymbol}
$


In [ ]:
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from ph121c_lxvm import models, data, basis, measure

## Many-body localized model

We introduce disorder to the model we have used before by allowing random
coefficients:
$$
    H = \sum_{j=1}^L \sigma_j^z \sigma_{j+1}^z
    - \sum_{j=1}^L h_j^x \sigma_j^x
    - \sum_{j=1}^L h_j^z \sigma_j^z 
    .
$$
The random coefficients $h_j^x$ and $h_j^z$ are sampled uniformly from $[W, W]$,
where the magnitude of $W$ determines the strength of the random noise.
This model may introduce localization of the quantum state, where the random
noise causes the probability mass to focus on some sites in the chain.

In [ ]:
%%time

save = '../../data/randoms.pick'

try:
    df = pd.read_pickle(save)
except FileNotFoundError:
    
    xi = np.array([-np.sqrt(3), 1]) * 0.5    
    dt = 0.05
    Nstp = 1000
    randoms = {
        's_0^?' : [],
        'vals' : [],
        'L' : [],
        'i' : [],
    }

    
    rng = np.random.default_rng(seed=935)
    W = 3.0
    Navg = 5
    bc = 'c'
    for L in [8, 10, 12]:
        psi = 1
        for i in range(L):
            psi = np.kron(xi, psi)
        for i in range(Navg):
            job = dict(
                oper=models.tfim_z.H_dense,
                oper_params={
                    'L' : L,
                    'h' : rng.uniform(low=-W, high=W, size=L),
                    'hz': rng.uniform(low=-W, high=W, size=L),
                    'bc': bc,
                },
                solver=np.linalg.eigh,
                solver_params={},
            )
            evals, evecs = data.jobs.obtain(**job)

            coef = evecs.T @ psi
            for which in ['x', 'y', 'z']:
                cevecs = (coef * evecs).T.astype('complex')
                tevals = np.exp(-1j*dt*evals)
                randoms['L'].append(L)
                randoms['i'].append(i)
                randoms['s_0^?'].append(which)
                randoms['vals'].append(
                    measure.evolve.Pauli_ev(
                        L=L, Nstp=Nstp, which=which, cevecs=cevecs, tevals=tevals,
                        num_threads=4
                    )
                )

    df = pd.DataFrame(randoms)
    df.to_pickle(save)

## Quantum many-body scar states

The following Hamiltonian is a toy model of a loop of spin-1/2 Rydberg atoms:
$$
    H = \frac{\Omega}{2} \sum_{j=1}^L \sigma_j^x
    + \sum_{j=1}^L P_{j, j+1} \sigma_{j+2}^z
    ,
$$
where
\begin{align}
    P_{j, j+1} 
        = (1 - \bm \sigma_j \cdot \bm \sigma_{j+1}) / 4
        = (1 - \sigma_j^x \sigma_{j+1}^x - \sigma_j^y \sigma_{j+1}^y - \sigma_j^z \sigma_{j+1}^z) / 4
    .
\end{align}
We are interested in this model because it contains scar states with unusually
low entanglement entropy. These can be viewed as the quantum analog of periodic
orbits in classically chaotic systems.

In [ ]:
%%time
save = '../../data/scars.pick'
try:
    df = pd.read_pickle(save)
except FileNotFoundError:

    scars = {
        'E' : [],
        'S' : [],
        'O' : [],
        'L' : [],
    }

    for L, O in product([8, 10, 12], [0.0, 0.5, 1.0, 4.0]):

        job = dict(
            oper=models.scars.H_dense,
            oper_params={
                'L' : L,
                'O' : O,
            },
            solver=np.linalg.eigh,
            solver_params={},
        )
        evals, evecs = data.jobs.obtain(**job)

        scars['L'].append(L)
        scars['O'].append(O)
        scars['E'].append(evals)
        scars['S'].append([
            # Evaluate entanglement entropy with respect to a half-subssystem
            # To use a random subsystem instead of a contiguous one, use comments
            # rng = np.random.default_rng(seed=935)
            measure.entropy.entanglement(basis.schmidt.values(
                evecs[:, i], np.arange(L//2), L
            #     evecs[i], rng.choice(np.arange(L), size=L//2, replace=False), L
            )) for i in range(evals.size)
        ])
        df = pd.DataFrame(scars)
        df.to_pickle(save)

In [ ]:
nrow = 4
ncol = 3
fig, axes = plt.subplots(nrow, ncol, sharey=True)
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        ax.scatter(
            df.E[i + j*nrow] / df.L[i + j*nrow],
            df.S[i + j*nrow] / df.L[i + j*nrow],
            s=20, marker='x', alpha=0.4,
        )
        ax.set_title('$L=$' + str(df.L[i + j*nrow]) \
            + ', $\Omega=$' + str(df.O[i + j*nrow]))
        ax.set_ylabel('$S_{L/2}/L$')
        ax.set_xlabel('$\lambda/L$')
        ax.vlines((df.O[i + j*nrow] / df.L[i + j*nrow]) \
            * (np.arange(df.L[i + j*nrow] + 1) - df.L[i + j*nrow] / 2),
            ymin=0, ymax=0.3, linestyle='dotted',
        )
fig.set_size_inches(9, 9)
fig.tight_layout()


## Code snippets

These are some useful code snippets I though I would drop here.
I had made a few mistakes while indexing for calculations, so I though I'd
give some code credit where it is due:

```python
# Verify orthogonality of eigenvectors (this takes a while)
for i, j in product(np.arange(evals.size), repeat = 2):
    if i > j:
        continue
    elif i == j:
        kron = 1
    else:
        kron = 0
    assert np.allclose(kron, np.inner(evecs[:, i].conj(), evecs[:, j])), str(i) + ' ' + str(j)
```

```python
# Verify eigenpairs
for i in range(evals.size):
    assert np.allclose(models.scars.H_vec(evecs[:, i], L, O), evals[i] * evecs[:, i]), str(i)
```